In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='met-museum-euro-artworks')])

In [32]:
from qdrant_client import models

def multi_stage_search(query: str, limit: int = 5) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name="met-museum-euro-artworks",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [41]:
def build_prompt(query, search_results):
    
    context = ""

    for doc in search_results:
        print(doc)
        context = context + f"artwork_description: {doc.payload['artwork_text']}\nimage_url: {doc.payload['artwork_image_url']}\n\n"

    prompt = f"""
        You are an AI assistant that answers user questions about artworks in the European Paintings collection at the Metropolitan Museum of Art. 
        You will be given context information from the museum's knowledge base. Use ONLY this context to answer the user's question. 
        If the answer cannot be found in the context, say "I could not find that information in the collection. Could you elaborate further on your question?" and then explain what is confusing you about the question. 
        Do not hallucinate or make up facts.

        Context:
        {context}

        User Question:
        {query}

        Instructions:
        - Answer in clear, concise, and user-friendly language. 
        - If available, include the title, artist, date, medium, and link to the official museum page. 
        - If images are provided in the context, return their URLs so they can be displayed. 
        - Keep the response factual and grounded in the provided context.

""".strip()
    
    return prompt

    

In [34]:
from openai import OpenAI

openai_client = OpenAI()
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [35]:
def rag(query):
    search_results = multi_stage_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
rag("Is there any paintings in the museum about Jerusalem?")

'Yes, there is a painting related to Jerusalem in the collection of the Metropolitan Museum of Art. It is titled "The Sack of Jerusalem by the Romans" by François Joseph Heim. \n\nHere are the details:\n\n- **Title:** The Sack of Jerusalem by the Romans\n- **Artist:** François Joseph Heim\n- **Date:** 1824\n- **Medium:** Oil on canvas\n- **Dimensions:** 14 x 15 in. (35.6 x 38.1 cm)\n- **Acquisition:** Catharine Lorillard Wolfe Collection, acquired in 2002\n\nCurrently, the painting is not showcased at the museum, but it is a significant artwork by Heim and is in the public domain.\n\nFor more information, you can visit the [official museum page](https://images.metmuseum.org/CRDImages/ep/original/DP-17607-001.jpg).\n\n![The Sack of Jerusalem by the Romans](https://images.metmuseum.org/CRDImages/ep/original/DP-17607-001.jpg)'

In [36]:
rag("I like oil on canvas paintings, I want recommendations of any oil on canvas paintings at the museum")

'Here are some oil on canvas paintings from the European Paintings collection at the Metropolitan Museum of Art:\n\n1. **La Grenouillère**  \n   - **Artist:** Claude Monet  \n   - **Date:** 1869  \n   - **Medium:** Oil on canvas  \n   - **Dimensions:** 29 3/8 x 39 1/4 in. (74.6 x 99.7 cm)  \n   - **Gallery:** 818  \n   - **Link:** [View La Grenouillère](https://maps.metmuseum.org/poi?_gl=1%2A1958ftb%2A_ga%2AMjk2MzAzMzczLjE3MDE4NzY3NzM.%2A_ga_Y0W8DGNBTB%2AMTcwODk4Mjg3Ny4yNDcuMS4xNzA4OTgyODgxLjAuMC4w&feature=f9f32fed0cb763c71a4c6bba3ad34ad3&floor=2&screenmode=base&search=405#19/40.7786001/-73.9640204/-61)  \n   - ![La Grenouillère](https://images.metmuseum.org/CRDImages/ep/original/DT1999.jpg)\n\n2. **Edmond Cavé (1794–1852)**  \n   - **Artist:** Jean Auguste Dominique Ingres  \n   - **Date:** 1844  \n   - **Medium:** Oil on canvas  \n   - **Dimensions:** 16 x 12 7/8 in. (40.6 x 32.7 cm)  \n   - **Gallery:** 801  \n   - **Link:** [View Edmond Cavé](https://maps.metmuseum.org/?feature=LTc

In [42]:
display(rag("Are there any paintings on display today? If so, in which galleries?"))

id=436548 version=0 score=4.630063 payload={'artwork_text': "Majas on a Balcony by Goya (Francisco de Goya y Lucientes). The description of the artwork is: 'The women known as majas visually distinguished themselves through opulent, exaggerated traditional dress that became synonymous with Spanish popular culture. Goya’s innovative composition of majas on a balcony seen from the street—accompanied by somewhat threatening male companions—was one of his most well-known paintings and is today in a private collection. This version may be a variant that Goya used to explore different expressive and stylistic emphases, or it may have been painted by a close follower. Goya’s complex compositional device, in which balcony and picture plane overlap on the brink of public and private spaces, would inspire French painter Edouard Manet in his depictions of urban life in Paris during the late 1860s.'. The source/origin of the artwork is H. O. Havemeyer Collection, Bequest of Mrs. H. O. Havemeye, th

'Yes, there are paintings on display today at the Metropolitan Museum of Art. Here are a few:\n\n1. **Majas on a Balcony**  \n   - **Artist:** Francisco de Goya y Lucientes  \n   - **Date:** Started in 1800 and completed in 1810  \n   - **Medium:** Oil on canvas  \n   - **Gallery:** 641  \n   - **Image:** ![Majas on a Balcony](https://images.metmuseum.org/CRDImages/ep/original/DP-20750-001.jpg)  \n   - [View on the Met website](https://www.metmuseum.org/art/collection/search/403)\n\n2. **Still Life with Silver**  \n   - **Artist:** Alexandre François Desportes  \n   - **Date:** Started in 1720 and completed in 1729  \n   - **Medium:** Oil on canvas  \n   - **Gallery:** 629  \n   - **Image:** ![Still Life with Silver](https://images.metmuseum.org/CRDImages/ep/original/DP283310.jpg)  \n   - [View on the Met website](https://www.metmuseum.org/art/collection/search/403)\n\n3. **Saint Francis of Assisi**  \n   - **Artist:** Antoniazzo Romano (Antonio di Benedetto Aquilio)  \n   - **Date:** 